# HW_1
*by Vasyukevich Andrey*

### Imports

In [1]:
import numpy as np
import pandas as pd
import nltk 
import sklearn
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
%matplotlib inline

1.
Загрузите датасет по ссылке: https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
(описание датасета можно посмотреть по ссылке
https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection )

2.
Считайте датасет в Python (можете сразу грузить все в память, выборка
небольшая), выясните, что используется в качестве разделителей и как
проставляются метки классов.

3.
Подготовьте для дальнейшей работы два списка: список текстов в порядке
их следования в датасете и список соответствующих им меток классов. В
качестве метки класса используйте 1 для спама и 0 для "не спама".

In [2]:
with open('readme') as readme:
    print readme.read()
    readme.close

SMS Spam Collection v.1
-------------------------

1. DESCRIPTION
--------------

The SMS Spam Collection v.1 (hereafter the corpus) is a set of SMS tagged messages that have been collected for SMS Spam research. It contains one set of SMS messages in English of 5,574 messages, tagged acording being ham (legitimate) or spam. 

1.1. Compilation
----------------

This corpus has been collected from free or free for research sources at the Web:

- A collection of between 425 SMS spam messages extracted manually from the Grumbletext Web site. This is a UK forum in which cell phone users make public claims about SMS spam messages, most of them without reporting the very spam message received. The identification of the text of spam messages in the claims is a very hard and time-consuming task, and it involved carefully scanning hundreds of web pages. The Grumbletext Web site is: http://www.grumbletext.co.uk/
- A list of 450 SMS ham messages collected from Caroline Tag's PhD Theses available 

In [3]:
dataset = open('SMSSpamCollection')
dataset_lines = filter(None, dataset.read().split('\n'))

In [4]:
dataset_lines[0]

'ham\tGo until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [5]:
labels, texts = zip(*map(lambda line: line.split('\t', 1), dataset_lines))

In [6]:
labels = np.asarray(map(lambda label: (label == 'spam'), labels))

In [7]:
texts = np.asarray(texts)

4.
Используя sklearn.feature_extraction.text.CountVectorizer со стандартными
настройками, получите из списка текстов матрицу признаков X.

In [8]:
count_vectorizer = CountVectorizer()

In [9]:
X_data = count_vectorizer.fit_transform(texts)

5.
Оцените качество классификации текстов с помощью LogisticRegression() с
параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое
качества на отдельных fold'ах. Параметр cv задайте равным 10. В качестве
метрики качества используйте f1-меру класса 1 (то есть, без микро и макро
усреднения). Получившееся качество – ответ в этом пункте.

In [10]:
estimator = LogisticRegression()

In [11]:
score = cross_val_score(estimator, X_data, labels, scoring='f1', cv=10)

In [12]:
np.mean(score.flat)

0.93334852685794145

6.
А теперь обучите классификатор на всей выборке и спрогнозируйте с его
помощью класс для следующих сообщений:
"FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"
"FreeMsg: Txt: claim your reward of 3 hours talk time"
"Have you visited the last lecture on physics?"
"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"
"Only 99$"
Выпишите через пробел прогнозы классификатора (0 – не спам, 1 – спам)

In [13]:
estimator = LogisticRegression()
estimator.fit(X_data, labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [14]:
messages = [
    "FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
    "FreeMsg: Txt: claim your reward of 3 hours talk time",
    "Have you visited the last lecture on physics?",
    "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
    "Only 99$"
]

In [15]:
print ' '.join(map(lambda b: str(int(b)), estimator.predict(count_vectorizer.transform(messages))))

1 1 0 0 0


7.
Задайте в CountVectorizer параметр ngram_range=(2,2), затем
ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте
получившееся в кросс-валидации значение f1-меры, округлите до второго
знака после точки, и выпишете результаты через пробел в том же порядке.
В данном эксперименте мы пробовали добавлять в признаки n-граммы для
разных диапазонов n - только биграммы, только триграммы, и, наконец, все
вместе - униграммы, биграммы и триграммы. Обратите внимание, что
статистики по биграммам и триграммам намного меньше, поэтому
классификатор только на них работает хуже. В то же время это не ухудшает
результат сколько-нибудь существенно, если добавлять их вместе с
униграммами, т.к. за счет регуляризации линейный классификатор не
склонен сильно переобучаться на этих признаках.

In [16]:
def cv_ngram_test(args, texts, labels, estimator, Vectorizer=CountVectorizer):
    count_vectorizer = Vectorizer(**args)
    X_data = count_vectorizer.fit_transform(texts)
    score = np.mean(cross_val_score(estimator, X_data, labels, scoring='f1', cv=10).flat)
    return score

In [17]:
args_list = [ { 'ngram_range': (2, 2) },  { 'ngram_range': (3, 3) },  { 'ngram_range': (1, 3) } ]

In [18]:
scores = [cv_ngram_test(args, texts, labels, LogisticRegression()) for args in args_list]

In [19]:
print ' '.join(map(lambda score: '%.2f' % score, scores))

0.82 0.73 0.93


8.
Повторите аналогичный п.7 эксперимент, используя вместо логистической
регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по
сравнению с линейным классификатором) наивный Байес страдает от
нехватки статистики по биграммам и триграммам.

In [20]:
cv_ngram_test({}, texts, labels, MultinomialNB())

0.92709424554224729

In [21]:
scores = [cv_ngram_test(args, texts, labels, MultinomialNB()) for args in args_list]

In [22]:
print ' '.join(map(lambda score: '%.2f' % score, scores))

0.65 0.38 0.89


9.
Попробуйте использовать в логистической регрессии в качестве признаков
Tf*idf из TfidfVectorizer на униграммах (рекомендуется использовать вместе
с sklearn.pipeline.Pipeline). Повысилось или понизилось качество на
кросс-валидации по сравнению с CountVectorizer на униграммах? Обратите
внимание, что результат перехода к tf*idf не всегда будет таким - если вы
наблюдаете какое-то явление на одном датасете, не надо сразу же его
обобщать на любые данные.

In [23]:
cv_ngram_test({}, texts, labels, LogisticRegression(), CountVectorizer)

0.93334852685794145

In [24]:
cv_ngram_test({}, texts, labels, LogisticRegression(), TfidfVectorizer)

0.85285995541724557

In [25]:
# Качество понизилось по сравнению с CountVectorizer

10.
Попробуйте получить как можно более высокое качество на
кросс-валидации. Обязательно уделите внимание подбору
макропараметров классификаторов (для этого используйте
sklearn.model_selection.GridSearchCV). Напишите, что пробовали и какое
качество получилось.

###### Потестируем RandomForestClassifier:

In [26]:
cv_ngram_test({}, texts, labels, RandomForestClassifier(), CountVectorizer)

0.89003150314339818

In [27]:
cv_ngram_test({}, texts, labels, RandomForestClassifier(), TfidfVectorizer)

0.89179309757439262

In [28]:
# Для RandomForestClassifier TfidfVectorizer показал чуть лучшие результаты!

In [29]:
cv_ngram_test({}, texts, labels, RandomForestClassifier(n_estimators=50, max_features='sqrt'), TfidfVectorizer)

0.90902779755333418

###### Попробуем перебирать параметры для LogisticRegression

In [30]:
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': np.logspace(-2, 5, num=30),
    'class_weight': ['balanced'],
    'max_iter': [100, 1000, 10000]
}

In [31]:
gscv = GridSearchCV(estimator=LogisticRegression(), param_grid=param_grid, scoring='f1', cv=10)

In [32]:
gscv.fit(X_data, labels)

GridSearchCV(cv=10, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': array([  1.00000e-02,   1.74333e-02,   3.03920e-02,   5.29832e-02,
         9.23671e-02,   1.61026e-01,   2.80722e-01,   4.89390e-01,
         8.53168e-01,   1.48735e+00,   2.59294e+00,   4.52035e+00,
         7.88046e+00,   1.37382e+01,   2.39503e+01,   4.1...         5.73615e+04,   1.00000e+05]), 'max_iter': [100, 1000, 10000], 'class_weight': ['balanced']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='f1', verbose=0)

In [33]:
est = gscv.best_estimator_

In [34]:
gscv.best_params_

{'C': 41.753189365604044,
 'class_weight': 'balanced',
 'max_iter': 100,
 'penalty': 'l2'}

In [35]:
print cv_ngram_test({}, texts, labels, est)

0.952030378413


In [36]:
# Наилучшую точность 0.952 дала LogisticRegression, судя по всему на улучшение точности повлияла установка
# class_weight='balanced' и оптимизация параметра регуляризации C.

11.
Какие наблюдения и выводы можно сделать из этого задания?

Обучение по сути велось только по ключевым словам, не задействуя синтаксическую структуру и n-граммы.
И по такому набору удалось достичь точности 0.952, следовательно, подтверждается естественная гипотеза,
что спам обладает некототорой характерной лексикой.